In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [8]:
import numpy as np
import argparse
import ast
import time
import matplotlib.pyplot as plt
import logging
import sys  
sys.path.insert(0, '../lib')

logging.basicConfig(level=logging.INFO)

In [74]:
from utils import *

In [160]:
%run ../lib/mple_learn_lib.ipynb

In [26]:
# current directory should be '/Users/hangjianli/Documents/research/change_point/python'
outdir = "../../output/scenario1" 

# Load setup info and data

In [27]:
logging.info("Loading network statistics and edge data...")
experiment_dir = outdir + ('' if outdir.endswith('/') else '/')

with open(experiment_dir + 'args/args.txt', 'r') as f: # comes from make_data.py
    sim_args_str = f.read()

sim_args = ast.literal_eval(sim_args_str)
p = len(sim_args['form_terms']) + len(sim_args['diss_terms'])

H = np.loadtxt(experiment_dir + sim_args['data_name'] + '_H.txt') # suff stats
y = np.loadtxt(experiment_dir + sim_args['data_name'] + '_y.txt') # networks

t = H.shape[0]
H = H.reshape((t, -1, p)) # t x n^2(E) x p
n = np.sqrt(H.shape[1]).astype(int)
logging.info(f"Data has dimension (t, n, p): ({t}, {n}, {p})")

INFO:root:Loading network statistics and edge data...
INFO:root:Data has dimension (t, n, p): (150, 20, 4)


array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 1., 1., 0.]])

In [50]:
sim_args

{'outdir': '../output/scenario1',
 'data_name': 'scenario1',
 'verbose': 1,
 'form_terms': ['edges', 'mutual'],
 'diss_terms': ['edges', 'mutual']}

In [165]:
args = {
#     'outdir': "../../output/test_new_gfl",
    'lam': 1,
    'admm_alpha': 10,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
#     'true_cp': None, #True change points (for plot).
#     "cp_thr": 3, #"Threshold for estimating change points
    "verbose": 1,
}


### Save args and define output files

In [39]:
# Get the output filenames
result_dir = make_directory(experiment_dir, 'results')
args_dir = make_directory(experiment_dir, 'args')

In [44]:
# save_args(args, args_dir + 'args_model.txt')

theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff.png'

### Run STERGM model

In [166]:
%run ../lib/mple_learn_lib.ipynb
%run ../lib/gfl_lib.ipynb

In [171]:
print('Initialize STERGM model...')
model = STERGMGraph(
    **args,
)

model.load_data(H, y, t, p)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=args["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)


INFO:root:<class 'float'>


Initialize STERGM model...
Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 0.19913315773010254.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.005316972732543945.
[INFO] Updating u...
[INFO] max mu :  0.8976811879405904
[INFO] dual_resnorm: 0.489936
[INFO] primal_resnorm: 0.196848
[INFO] convergence: 0.489936
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 0.17455482482910156.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0043261051177978516.
[INFO] Updating u...
[INFO] max mu :  0.905372694366374
[INFO] dual_resnorm: 0.150757
[INFO] primal_resnorm: 0.105615
[INFO] convergence: 0.150757
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 0.13983607292175293.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.006646156311035156.
[INFO] Updating u..

[INFO] Newton converged in: 0.10501289367675781.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0040531158447265625.
[INFO] Updating u...
[INFO] max mu :  0.9692918077708352
[INFO] dual_resnorm: 0.046772
[INFO] primal_resnorm: 0.046849
[INFO] convergence: 0.046849
-------------------------------------------------
[INFO] ADMM step #23
[INFO] Updating theta...
[INFO] Newton converged in: 0.13187885284423828.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0077631473541259766.
[INFO] Updating u...
[INFO] max mu :  0.9463076312675314
[INFO] dual_resnorm: 0.014514
[INFO] primal_resnorm: 0.008331
[INFO] convergence: 0.014514
-------------------------------------------------
[INFO] ADMM step #24
[INFO] Updating theta...
[INFO] Newton converged in: 0.11584711074829102.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.012303829193115234.
[INFO] Updating u...
[INFO] max mu :  0.9416332315817226
[INFO] dual_resnorm: 0.007032
[INFO] primal_resnorm: 0.00360

[INFO] Group fused Lasso converged in: 0.012404918670654297.
[INFO] Updating u...
[INFO] max mu :  0.9343789980414721
[INFO] dual_resnorm: 0.000015
[INFO] primal_resnorm: 0.000013
[INFO] convergence: 0.000015
-------------------------------------------------
[INFO] ADMM step #47
[INFO] Updating theta...
[INFO] Newton converged in: 0.05270814895629883.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.010926246643066406.
[INFO] Updating u...
[INFO] max mu :  0.9343811615659658
[INFO] dual_resnorm: 0.000013
[INFO] primal_resnorm: 0.000010
[INFO] convergence: 0.000013
-------------------------------------------------
[INFO] ADMM step #48
[INFO] Updating theta...
[INFO] Newton converged in: 0.051223039627075195.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.011233091354370117.
[INFO] Updating u...
[INFO] max mu :  0.9343830040826941
[INFO] dual_resnorm: 0.000011
[INFO] primal_resnorm: 0.000008
[INFO] convergence: 0.000011
----------------------------------------

In [172]:
%run ../lib/report_lib.ipynb

In [173]:
plot_theta(
    theta[1:], 
    theta_plot_dir,
    thr=0.3,
    change_pts=None
)

The estimated change points are [ 50 100]
